In [ ]:
import json
import requests
from folium import Map, TileLayer, GeoJson

endpoint = ""

In [ ]:
geojson = {"type": "Feature", "geometry": {"coordinates": [[[-87.0251, 36.2251], [-87.0251, 36.0999], [-85.4249, 36.0999], [-85.4249, 36.2251], [-87.0251, 36.2251]]], "type": "Polygon"}}
bounds = (-87.0251, 36.0999, -85.4249, 36.2251)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)
m

### Create Mosaic for the whole collection

In [ ]:
# Register Search Query
body = {
    "collections": ["noaa-emergency-response"],
    # bbox is optional but helps tilejson to restric the bounding box of the mosaic
    "bbox": bounds,
}

response = requests.post(
    f"{endpoint}/mosaic/register",
    json=body,
).json()
print(response)

In [ ]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

# Fetch Tilejson (we HAVE TO add the asset name)
tj_resp = requests.get(
    response['url'],
    params={
        # Info to add to the tilejson response
        "minzoom": 13,
        "maxzoom": 20,
        # query parameter to add to the tile URL
        "assets": "cog",
    }
).json()
print(tj_resp)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=tj_resp["tiles"][0],
    attr="Mosaic",
    min_zoom=13,
    max_zoom=20,
    max_native_zoom=20,    
)
aod_layer.add_to(m)
m